<a href="https://colab.research.google.com/github/thaisgulias/DataAnalytics_student/blob/main/Exercicios_Comandos_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=61ca6740ad8ce80f461b94bc70dedd29bf9c17797d1a418a2a527e70c5e19418
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from pyspark.sql.types import IntegerType, FloatType, DoubleType

In [ ]:
spark = (SparkSession.builder
                     .master('local')
                     .appName('spotify')
                     .config('spark.ui.port', '4050')
                     .getOrCreate())

In [ ]:
df = (spark.read
           .format('csv')
           .option('delimiter', ',')
           .option('header', 'true')
           .option('inferschema', 'true')
           .option('encoding', 'utf-8')
           .option('escape','"')
           .load('/content/drive/MyDrive/SOULCODE/spotify .csv')
           )

In [ ]:
print(f'({df.count()}, {len(df.columns)})')

(18835, 16)


In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: double (nullable = true)



In [ ]:
df.show(truncate=False)

+---+--------------------------+---------------+----------------+----------------------+-----------------------+------------------+---------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|_c0|song_name                 |song_popularity|song_duration_ms|acousticness          |danceability           |energy            |instrumentalness     |key |liveness           |loudness           |audio_mode|speechiness         |tempo  |time_signature|audio_valence     |
+---+--------------------------+---------------+----------------+----------------------+-----------------------+------------------+---------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|0  |Boulevard of Broken Dreams|73             |262333          |0.005520000000000001kg|0.496mol/L             |0.682             |2.94e-05             |8.0 |0.0589             |-4.

In [ ]:
#Quantidade de registros do dataframe
qtd_registros = df.count()

In [ ]:
df.drop(F.col('_c0')).filter(F.col('song_name') == 'TEST DRIVE').drop_duplicates().show(truncate=False)

+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+-----------------+--------------+-------------+
|song_name |song_popularity|song_duration_ms|acousticness|danceability           |energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo            |time_signature|audio_valence|
+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+-----------------+--------------+-------------+
|TEST DRIVE|71             |179423          |0.71kg      |0.6459999999999999mol/L|0.648 |0.00316         |3.0|0.108   |-8.602  |1         |0.0372     |75.00399999999998|4             |0.466        |
+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+-----------------+--------------+-------------+



In [ ]:
df.select('song_name').distinct().count()

13070

In [ ]:
#EXISTE ALGUM REGISTRO DUPLICADO?
qtd_sem_duplicados = df.drop(F.col('_c0')).drop_duplicates().count()

In [ ]:
total_duplicados = qtd_registros - qtd_sem_duplicados

In [ ]:
print(f'Total de linhas duplicadas: {total_duplicados}')

Total de linhas duplicadas: 3903


In [ ]:
df = df.drop(F.col('_c0'))

In [ ]:
#AQUI ESTAMOS REMOVENDOS VALORES DUPLICADOS, QUE SÃO INCONSISTÊNCIAS NO MEU DATASET
df = df.drop_duplicates()

In [ ]:
df.count()

14932

In [ ]:
df.show(truncate=False)

+---------------------------+---------------+----------------+-----------------------+------------------------+------------------+----------------------+----+-------------------+--------+----------+-----------+------------------+--------------+-------------------+
|song_name                  |song_popularity|song_duration_ms|acousticness           |danceability            |energy            |instrumentalness      |key |liveness           |loudness|audio_mode|speechiness|tempo             |time_signature|audio_valence      |
+---------------------------+---------------+----------------+-----------------------+------------------------+------------------+----------------------+----+-------------------+--------+----------+-----------+------------------+--------------+-------------------+
|Sk8er Boi                  |67             |204000          |6.790000000000001e-05kg|0.487mol/L              |0.9               |0.0                   |0.0 |0.358              |-4.417  |1         |0.0482 

In [ ]:
#VERIFICAR O SUMMARY
df.summary().show()

+-------+--------------------+------------------+--------------------+------------+------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+
|summary|           song_name|   song_popularity|    song_duration_ms|acousticness|danceability|             energy|   instrumentalness|               key|          liveness|           loudness|         audio_mode|        speechiness|             tempo|     time_signature|     audio_valence|
+-------+--------------------+------------------+--------------------+------------+------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+
|  count|               14932|             14932|               14932|       14932|       14932|              14932|     

In [ ]:
df.select(F.col('acousticness')).withColumn('acustica', F.regexp_replace('acousticness', 'kg', '')).show()

In [ ]:
#2ª INCONSISTÊNCIA - REMOVER O kg DA VARIÁVEL ACOUSTICNESS
df = df.withColumn('acustica', F.regexp_replace('acousticness', 'kg', ''))

In [ ]:
df = df.drop(F.col('acousticness'))

In [ ]:
df.show()

+--------------------+---------------+----------------+-------------------+------------------+--------------------+----+-------------------+--------+----------+-----------+------------------+--------------+-------------------+--------------------+
|           song_name|song_popularity|song_duration_ms|       danceability|            energy|    instrumentalness| key|           liveness|loudness|audio_mode|speechiness|             tempo|time_signature|      audio_valence|            acustica|
+--------------------+---------------+----------------+-------------------+------------------+--------------------+----+-------------------+--------+----------+-----------+------------------+--------------+-------------------+--------------------+
|           Sk8er Boi|             67|          204000|              0.487|               0.9|                 0.0| 0.0|              0.358|  -4.417|         1|     0.0482|           149.937|             4|              0.484|6.790000000000001...|
|  Whene

In [ ]:
#3ª INCONSISTÊNCIA - REMOVER O mol/L DA VARIÁVEL ACOUSTICNESS
df = df.withColumn('danceability', F.regexp_replace('danceability', 'mol/L', ''))

In [ ]:
df.select('song_popularity').summary().show()

+-------+------------------+
|summary|   song_popularity|
+-------+------------------+
|  count|             14932|
|   mean| 48.75768535262206|
| stddev|20.379617796732845|
|    min|                 0|
|    25%|              37.0|
|    50%|              52.0|
|    75%|              64.0|
|    max|           nao_sei|
+-------+------------------+



In [ ]:
#função que remove strings

def remove_strings_inplace(df, colunas):
    # itera sobre as colunas especificadas e aplica a transformação em cada uma
    for column in colunas:
        df = df.withColumn(column, F.regexp_replace(F.col(column), "[^0-9.eE-]", ""))
    return df
#chama a função passando de parâmetro o df e a lista criada

In [ ]:
colunas = df.drop(F.col('song_name')).columns

In [ ]:
df = remove_strings_inplace(df, colunas)

In [ ]:
#VERIFICAR OS não_sei DA COLUNA song_popularity
df.filter(F.col('instrumentalness').contains('nao')).show()

+--------------------+---------------+----------------+------------+------------------+------+----------------+----+--------+-------------------+----------+-----------+------+--------------+-------------+
|           song_name|song_popularity|song_duration_ms|acousticness|      danceability|energy|instrumentalness| key|liveness|           loudness|audio_mode|speechiness| tempo|time_signature|audio_valence|
+--------------------+---------------+----------------+------------+------------------+------+----------------+----+--------+-------------------+----------+-----------+------+--------------+-------------+
|Love Will Tear Us...|             60|          223340|     0.951kg|             0.375| 0.217|         nao_sei| 3.0|   0.107|-12.425999999999998|         1|     0.0344|96.239|             3|        0.139|
|     Sunny Came Home|             62|          264200|     0.344kg|0.5539999999999999|  0.58|         nao_sei|11.0|  0.0989|              -8.05|         0|     0.0322|167.83|     

In [ ]:
df = df.replace('nao_sei', None)
df = df.replace('0.nao_sei', None)
df = df.dropna(how='any')

In [ ]:
#VERIFICAR A QUANTIDADE DE VALORES NULOS NO DATAFRAME
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
df.select(F.col('instrumentalness')).summary().show()

+-------+-------------------+
|summary|   instrumentalness|
+-------+-------------------+
|  count|              14919|
|   mean|0.09204778446611722|
| stddev| 0.2404034647567487|
|    min|                0.0|
|    25%|                0.0|
|    50%|            2.08E-5|
|    75%|            0.00511|
|    max|              9e-06|
+-------+-------------------+



In [ ]:
df.select(F.col('instrumentalness')).orderBy(F.col('instrumentalness').desc()).show(500)

+--------------------+
|    instrumentalness|
+--------------------+
|               9e-06|
|               9e-05|
|9.990000000000003...|
|            9.98e-06|
|            9.98e-06|
|            9.98e-05|
|            9.97e-05|
|            9.96e-05|
|            9.94e-06|
|            9.94e-06|
|9.940000000000003...|
|            9.93e-06|
|            9.92e-05|
|             9.8e-06|
|             9.8e-06|
|             9.8e-05|
|            9.88e-05|
|            9.86e-06|
|            9.86e-06|
|            9.86e-06|
|            9.86e-05|
|            9.86e-05|
|            9.85e-05|
|            9.84e-06|
|            9.84e-06|
|            9.84e-05|
|            9.82e-06|
|            9.81e-05|
|            9.78e-06|
|            9.78e-05|
|            9.77e-05|
|            9.75e-06|
|            9.75e-05|
|            9.75e-05|
|            9.73e-05|
|            9.73e-05|
|            9.72e-06|
|            9.71e-05|
|             9.6e-06|
|             9.6e-06|
|          

In [ ]:
df.select(F.col('instrumentalness')).withColumn('teste', F.col('instrumentalness').cast(DoubleType())).printSchema()

root
 |-- instrumentalness: string (nullable = true)
 |-- teste: double (nullable = true)



In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)
 |-- acustica: string (nullable = true)



In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)
 |-- acustica: string (nullable = true)



In [ ]:
#CONVERTER AS COLUNAS PARA INT E FLOAT
df = (df.withColumn('song_popularity', F.col('song_popularity').cast(IntegerType()))
   .withColumn('song_duration_ms', F.round(F.col('song_duration_ms').cast(IntegerType()), 2))
   .withColumn('danceability', F.round(F.col('danceability').cast(FloatType()), 2))
   .withColumn('energy', F.round(F.col('energy').cast(FloatType()), 2))
   .withColumn('instrumentalness', F.round(F.col('instrumentalness').cast(FloatType()), 2))
   .withColumn('key', F.round(F.col('key').cast(FloatType()), 2))
   .withColumn('liveness', F.round(F.col('liveness').cast(FloatType()), 2))
   .withColumn('loudness', F.round(F.col('loudness').cast(FloatType()), 2))
   .withColumn('audio_mode', F.round(F.col('audio_mode').cast(FloatType()), 2))
   .withColumn('speechiness', F.round(F.col('speechiness').cast(FloatType()), 2))
   .withColumn('tempo', F.round(F.col('tempo').cast(IntegerType()), 2))
   .withColumn('time_signature', F.round(F.col('time_signature').cast(IntegerType()), 2))
   .withColumn('audio_valence', F.round(F.col('audio_valence').cast(FloatType()), 2))
   .withColumn('acustica', F.round(F.col('acustica').cast(FloatType()), 2))
)

In [ ]:
df.show()

+--------------------+---------------+----------------+------------+------+----------------+----+--------+--------+----------+-----------+-----+--------------+-------------+--------+
|           song_name|song_popularity|song_duration_ms|danceability|energy|instrumentalness| key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|acustica|
+--------------------+---------------+----------------+------------+------+----------------+----+--------+--------+----------+-----------+-----+--------------+-------------+--------+
|           Sk8er Boi|             67|          204000|        0.49|   0.9|             0.0| 0.0|    0.36|   -4.42|       1.0|       0.05|  149|             4|         0.48|     0.0|
|  Whenever, Wherever|             76|          196160|        0.79|  0.83|             0.0| 1.0|     0.2|   -4.86|       0.0|       0.04|  107|             4|         0.87|    0.24|
|        Paloma negra|             50|          195533|        0.39|  0.26|          

In [ ]:
df.summary().show()

+-------+--------------------+------------------+------------------+------------------+-------------------+-------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
|summary|           song_name|   song_popularity|  song_duration_ms|      danceability|             energy|   instrumentalness|              key|           liveness|          loudness|        audio_mode|        speechiness|             tempo|    time_signature|      audio_valence|           acustica|
+-------+--------------------+------------------+------------------+------------------+-------------------+-------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
|  count|               14932|             14931|             14931|             14932|       

In [ ]:
df.select(F.col('song_popularity')).filter(F.col('song_popularity').contains('')).show()

+---------------+
|song_popularity|
+---------------+
|             67|
|             76|
|             50|
|             71|
|             84|
|             34|
|             58|
|             32|
|             59|
|             65|
|             52|
|             41|
|             81|
|             23|
|             30|
|             60|
|             58|
|             64|
|             49|
|             52|
+---------------+
only showing top 20 rows

